In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
from keras_tuner import RandomSearch, Hyperband, HyperParameters

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from datetime import date
import warnings
warnings.filterwarnings('ignore')
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
os.makedirs('LSTM_Models', exist_ok=True)
os.makedirs('LSTM_Scalers', exist_ok=True)

def clean_filename(name):
    import re
    return re.sub(r'[^A-Za-z0-9]+', '_', name)

In [ ]:
import tensorflow as tf
import keras

In [ ]:
print(tf.__version__)
print(keras.__version__)

2.18.0
3.8.0


In [ ]:
df = pd.read_csv('/content/df_Prophet_Autots.csv')

In [ ]:
df.head()

,Time Serie,AUSTRALIA - AUSTRALIAN DOLLAR/US$,EURO AREA - EURO/US$,NEW ZEALAND - NEW ZELAND DOLLAR/US$,UNITED KINGDOM - UNITED KINGDOM POUND/US$,BRAZIL - REAL/US$,CANADA - CANADIAN DOLLAR/US$,CHINA - YUAN/US$,HONG KONG - HONG KONG DOLLAR/US$,INDIA - INDIAN RUPEE/US$,...,SINGAPORE - SINGAPORE DOLLAR/US$,DENMARK - DANISH KRONE/US$,JAPAN - YEN/US$,MALAYSIA - RINGGIT/US$,NORWAY - NORWEGIAN KRONE/US$,SWEDEN - KRONA/US$,SRI LANKA - SRI LANKAN RUPEE/US$,SWITZERLAND - FRANC/US$,TAIWAN - NEW TAIWAN DOLLAR/US$,THAILAND - BAHT/US$
0,2000-01-03,1.5172,0.9847,1.9033,0.6146,1.8050,1.4465,8.2798,7.7765,43.55,...,1.6563,7.3290,101.70,3.8,7.964,8.4430,72.30,1.5808,31.38,36.97
1,2000-01-04,1.5239,0.9700,1.9238,0.6109,1.8405,1.4518,8.2799,7.7775,43.55,...,1.6535,7.2180,103.09,3.8,7.934,8.3600,72.65,1.5565,30.60,37.13
2,2000-01-05,1.5267,0.9676,1.9339,0.6092,1.8560,1.4518,8.2798,7.7780,43.55,...,1.6560,7.2080,103.77,3.8,7.935,8.3530,72.95,1.5526,30.80,37.10
3,2000-01-06,1.5291,0.9686,1.9436,0.6070,1.8400,1.4571,8.2797,7.7785,43.55,...,1.6655,7.2125,105.19,3.8,7.940,8.3675,72.95,1.5540,31.75,37.62
4,2000-01-07,1.5272,0.9714,1.9380,0.6104,1.8310,1.4505,8.2794,7.7783,43.55,...,1.6625,7.2285,105.17,3.8,7.966,8.4150,73.15,1.5623,30.85,37.30


In [ ]:
# Initializing MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
#Initialize the size of the test set
test_size = 60

In [ ]:
# split the data into train, test, val before scaling
train_end = -test_size - int(0.3 * (len(df) - test_size))  # 70% train, 30% val
val_end = -test_size


In [ ]:
train = df.iloc[:train_end]
val = df.iloc[train_end:val_end]
test = df.iloc[val_end:]

In [ ]:
scaler = MinMaxScaler()

# Fit ONLY on training data, define the columns to scale
columns_to_scale = df.columns.difference(['Time Serie'])

In [ ]:
#Disctionary for storing scalers for later use
scalers = {}



In [ ]:
#Fit the scaler on the train data
for col in columns_to_scale:
    scaler = MinMaxScaler()
    scaler.fit(train[[col]])
    scalers[col] = scaler

In [ ]:

train_scaled = train.copy()
val_scaled = val.copy()
test_scaled = test.copy()


In [ ]:
# Transform all sets using training scaler
for col in columns_to_scale:
    train_scaled[col] = scalers[col].transform(train[[col]])
    val_scaled[col] = scalers[col].transform(val[[col]])
    test_scaled[col] = scalers[col].transform(test[[col]])

In [ ]:
def create_uni_sequences(data, input_length):
    X, y = [], []
    for i in range(len(data) - input_length):
        X.append(data[i:i+input_length])
        y.append(data[i+input_length])
    return np.array(X), np.array(y)

In [ ]:
def build_lstm_model(hp, input_length):
    model = Sequential()
    num_layers = hp.Int('num_layers', 1, 3)
    for i in range(num_layers):
        units = hp.Int(f'units_{i}', 32, 128, step=32)
        return_sequences = (i < num_layers - 1)
        if i == 0:
            model.add(LSTM(units, return_sequences=return_sequences, input_shape=(input_length, 1)))
        else:
            model.add(LSTM(units, return_sequences=return_sequences))
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(hp.Float(f'dropout_rate_{i}', 0.1, 0.5, step=0.1)))
    model.add(Dense(1, activation='linear'))
    lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    return model

In [ ]:
for currency in columns_to_scale:
    print(f"\n🔥 Tuning and Training LSTM for {currency}")

    # 1. Prepare data
    currency_train = train[[currency]].values
    currency_val = val[[currency]].values
    currency_test = test[[currency]].values

    # 2. Fit scaler and save
    scaler = MinMaxScaler().fit(currency_train)
    joblib.dump(scaler, f'LSTM_Scalers/scaler_{clean_filename(currency)}.pkl')

    # 3. Scale data
    train_scaled = scaler.transform(currency_train)
    val_scaled = scaler.transform(currency_val)
    test_scaled = scaler.transform(currency_test)

    # 4. Create sequences
    X_train, y_train = create_uni_sequences(train_scaled, input_sequence_length)
    X_val, y_val = create_uni_sequences(val_scaled, input_sequence_length)
    X_test, y_test = create_uni_sequences(test_scaled, input_sequence_length)

    # 5. Hyperparameter tuning
    tuner = Hyperband(
        lambda hp: build_lstm_model(hp, input_sequence_length),
        objective='val_loss',
        max_epochs=30,
        factor=3,
        directory=f'lstm_tuning_{clean_filename(currency)}',
        project_name=f'forex_lstm_{clean_filename(currency)}',
        overwrite=True
    )
    tuner.search(X_train, y_train, epochs=30, validation_data=(X_val, y_val), callbacks=[early_stop], verbose=0)
    best_hps = tuner.get_best_hyperparameters(1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    # 6. Train best model
    best_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, callbacks=[early_stop], verbose=0)

    # 7. Save best model for this currency
    best_model.save(f'LSTM_Models/{clean_filename(currency)}_best_model.keras')


🔥 Tuning and Training LSTM for AUSTRALIA - AUSTRALIAN DOLLAR/US$

🔥 Tuning and Training LSTM for BRAZIL - REAL/US$

🔥 Tuning and Training LSTM for CANADA - CANADIAN DOLLAR/US$

🔥 Tuning and Training LSTM for CHINA - YUAN/US$

🔥 Tuning and Training LSTM for DENMARK - DANISH KRONE/US$

🔥 Tuning and Training LSTM for EURO AREA - EURO/US$

🔥 Tuning and Training LSTM for HONG KONG - HONG KONG DOLLAR/US$

🔥 Tuning and Training LSTM for INDIA - INDIAN RUPEE/US$

🔥 Tuning and Training LSTM for JAPAN - YEN/US$

🔥 Tuning and Training LSTM for KOREA - WON/US$

🔥 Tuning and Training LSTM for MALAYSIA - RINGGIT/US$

🔥 Tuning and Training LSTM for MEXICO - MEXICAN PESO/US$

🔥 Tuning and Training LSTM for NEW ZEALAND - NEW ZELAND DOLLAR/US$

🔥 Tuning and Training LSTM for NORWAY - NORWEGIAN KRONE/US$

🔥 Tuning and Training LSTM for SINGAPORE - SINGAPORE DOLLAR/US$

🔥 Tuning and Training LSTM for SOUTH AFRICA - RAND/US$

🔥 Tuning and Training LSTM for SRI LANKA - SRI LANKAN RUPEE/US$

🔥 Tuning and Tr

In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create a folder in the root directory
!mkdir -p "/content/drive/My Drive/LSTM FOLDER"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
import shutil

shutil.make_archive("LSTM_Models", 'zip', "LSTM_Models")
files.download("LSTM_Models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("LSTM_Models", 'zip', "LSTM_Models")
files.download("LSTM_Models.zip")

In [ ]:
!zip -r /content/LSTM_Models.zip /content/content/LSTM_Models